In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import kstest, weibull_min
import os

# Funkcja wczytująca dane z dokładną diagnostyką
def wczytaj_dane(plik_csv):
    try:
        # Wczytaj dane
        dane = pd.read_csv(plik_csv)
        print(f"Wczytano plik {plik_csv} z {len(dane)} wierszami")
        
        # Pokaż pierwsze kilka wierszy
        print("\nPierwsze 2 wiersze danych:")
        print(dane.head(2))
        
        # Sprawdź unikalne wartości dla Track_Completed
        if 'Track_Completed' in dane.columns:
            print("\nUnikalne wartości w kolumnie Track_Completed:")
            print(dane['Track_Completed'].unique())
            print(f"Typy wartości: {[type(x).__name__ for x in dane['Track_Completed'].unique()]}")
        
        # Sprawdź wartości dla Track_ID
        if 'Track_ID' in dane.columns:
            print("\nUnikalne wartości w kolumnie Track_ID:")
            print(sorted(dane['Track_ID'].unique()))
        
        return dane
    except Exception as e:
        print(f"Błąd wczytywania danych: {e}")
        return None

# Funkcja testująca dopasowanie rozkładu Weibulla
def test_weibull(dane_czasowe, nazwa_toru, grupa=None, typ_grupy=None, poziom_istotnosci=0.05):
    if len(dane_czasowe) < 3:
        grupa_info = f", {typ_grupy}: {grupa}" if grupa is not None else ""
        print(f"Za mało danych dla toru {nazwa_toru}{grupa_info} do przeprowadzenia testu.")
        return None
    
    dane_czasowe = np.array(dane_czasowe)
    
    # Estymacja parametrów rozkładu Weibulla metodą największej wiarygodności
    try:
        shape, loc, scale = weibull_min.fit(dane_czasowe, floc=0)
        
        # Test Kołmogorowa-Smirnova
        ks_statistic, p_value = kstest(dane_czasowe, 'weibull_min', args=(shape, loc, scale))
        
        # Wartość krytyczna dla testu KS
        critical_value = stats.ksone.ppf(1 - poziom_istotnosci / 2, len(dane_czasowe))
        
        # Czy dane mają rozkład Weibulla?
        is_weibull = ks_statistic < critical_value
        
        wyniki = {
            'tor': nazwa_toru,
            'grupa': grupa if grupa is not None else "wszystkie",
            'typ_grupy': typ_grupy if typ_grupy is not None else "ogólne",
            'liczba_probek': len(dane_czasowe),
            'shape': shape,
            'scale': scale,
            'ks_statistic': ks_statistic,
            'p_value': p_value,
            'critical_value': critical_value,
            'is_weibull': is_weibull
        }
        
        # Wydrukuj wyniki
        grupa_info = f", {typ_grupy}: {grupa}" if grupa is not None else ""
        print(f"\n--- Test rozkładu Weibulla dla toru {nazwa_toru}{grupa_info} ---")
        print(f"Liczba próbek: {len(dane_czasowe)}")
        print(f"Parametry rozkładu: kształt (shape)={shape:.4f}, skala (scale)={scale:.2f}")
        print(f"Statystyka testu K-S: {ks_statistic:.4f}")
        print(f"Wartość p: {p_value:.4f}")
        print(f"Wartość krytyczna (α={poziom_istotnosci}): {critical_value:.4f}")
        print(f"Wynik testu: {'Dane mają rozkład Weibulla' if is_weibull else 'Dane NIE mają rozkładu Weibulla'}")
        
        # Wizualizacja wyników
        katalog = "wykresy_weibulla"
        if typ_grupy is not None:
            katalog = f"{katalog}/tor_{nazwa_toru}/{typ_grupy}"
            os.makedirs(katalog, exist_ok=True)
            nazwa_pliku = f"{katalog}/tor_{nazwa_toru}_{typ_grupy}_{str(grupa).replace(' ', '_').replace('<', 'lt').replace('-', '_')}.png"
        else:
            katalog = f"{katalog}/tor_{nazwa_toru}"
            os.makedirs(katalog, exist_ok=True)
            nazwa_pliku = f"{katalog}/tor_{nazwa_toru}_wszystkie.png"
        
        wizualizuj_weibull(dane_czasowe, shape, scale, nazwa_toru, grupa, typ_grupy, nazwa_pliku)
        
        return wyniki
    except Exception as e:
        grupa_info = f", {typ_grupy}: {grupa}" if grupa is not None else ""
        print(f"Błąd podczas analizy Weibulla dla toru {nazwa_toru}{grupa_info}: {e}")
        return None

# Funkcja do wizualizacji dopasowania rozkładu Weibulla
# Funkcja do wizualizacji dopasowania rozkładu Weibulla
def wizualizuj_weibull(dane, shape, scale, nazwa_toru, grupa=None, typ_grupy=None, nazwa_pliku=None):
    plt.figure(figsize=(12, 10))

    tytul_dodatek = f", {typ_grupy}: {grupa}" if grupa is not None else ""

    # Plot 1: Histogram and PDF
    plt.subplot(2, 2, 1)
    counts, bins, _ = plt.hist(dane, bins=max(5, min(20, len(dane)//2)), density=True, alpha=0.7, label='Empirical data')
    x = np.linspace(min(dane), max(dane), 1000)
    pdf = weibull_min.pdf(x, shape, scale=scale)
    plt.plot(x, pdf, 'r-', lw=2, label=f'Weibull PDF (shape={shape:.2f}, scale={scale:.2f})')
    plt.title(f'Histogram and PDF - Track {nazwa_toru}{tytul_dodatek}')
    plt.xlabel('Time [ms]')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 2: Empirical and theoretical CDF
    plt.subplot(2, 2, 2)
    sorted_data = np.sort(dane)
    ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    plt.step(sorted_data, ecdf, label='Empirical CDF')
    cdf = weibull_min.cdf(x, shape, scale=scale)
    plt.plot(x, cdf, 'r-', lw=2, label='Weibull CDF')
    plt.title(f'Cumulative Distribution Function - Track {nazwa_toru}{tytul_dodatek}')
    plt.xlabel('Time [ms]')
    plt.ylabel('Cumulative probability')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 3: Q-Q plot
    plt.subplot(2, 2, 3)
    theoretical_quantiles = weibull_min.ppf(np.linspace(0.01, 0.99, len(dane)), shape, scale=scale)
    plt.scatter(np.sort(theoretical_quantiles), np.sort(dane))
    min_val = min(min(theoretical_quantiles), min(dane))
    max_val = max(max(theoretical_quantiles), max(dane))
    plt.plot([min_val, max_val], [min_val, max_val], 'k--')
    plt.title(f'Q-Q Plot - Track {nazwa_toru}{tytul_dodatek}')
    plt.xlabel('Theoretical quantiles')
    plt.ylabel('Empirical quantiles')
    plt.grid(True, alpha=0.3)

    # Plot 4: P-P plot
    plt.subplot(2, 2, 4)
    empirical_cdf = ecdf
    theoretical_cdf = weibull_min.cdf(sorted_data, shape, scale=scale)
    plt.scatter(empirical_cdf, theoretical_cdf)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'P-P Plot - Track {nazwa_toru}{tytul_dodatek}')
    plt.xlabel('Empirical probability')
    plt.ylabel('Theoretical probability')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    if nazwa_pliku:
        plt.savefig(nazwa_pliku)
    plt.close()


# Główna funkcja do przeprowadzenia testów
def wykonaj_testy_weibulla(plik_games):
    # Wczytaj dane
    dane = wczytaj_dane(plik_games)
    
    if dane is None:
        return
    
    # WAŻNE: Sprawdź, jak wygląda kolumna Track_Completed i odpowiednio ją przekształć
    if 'Track_Completed' in dane.columns:
        # Najpierw sprawdź, jakie są rzeczywiste wartości w danych
        print("\nWartości w kolumnie Track_Completed przed konwersją:")
        wartosci = dane['Track_Completed'].value_counts()
        print(wartosci)
        
        # Sprawdź, czy wartości są typu string
        if dane['Track_Completed'].dtype == 'object':
            print("Konwersja wartości tekstowych w Track_Completed...")
            # Konwersja 'True'/'False' na True/False
            dane['Track_Completed'] = dane['Track_Completed'].map({'True': True, 'False': False})
        # Jeśli wartości są liczbowe, konwertuj na boolean
        elif dane['Track_Completed'].dtype in ['int64', 'float64']:
            print("Konwersja wartości liczbowych w Track_Completed...")
            dane['Track_Completed'] = dane['Track_Completed'] > 0
    
    # Pokaż wyniki konwersji
    print("\nWartości w kolumnie Track_Completed po konwersji:")
    if 'Track_Completed' in dane.columns:
        print(dane['Track_Completed'].value_counts())
    
    # Filtruj tylko ukończone gry - bez zakładania, czy wartość to True czy 'True'
    ukonczone_gry = None
    if 'Track_Completed' in dane.columns:
        # Próba filtrowania z wartością True
        ukonczone_gry_true = dane[dane['Track_Completed'] == True]
        print(f"Liczba gier z Track_Completed == True: {len(ukonczone_gry_true)}")
        
        # Próba filtrowania z wartością 'True' (string)
        ukonczone_gry_string = dane[dane['Track_Completed'] == 'True']
        print(f"Liczba gier z Track_Completed == 'True' (string): {len(ukonczone_gry_string)}")
        
        # Wybierz lepszą opcję
        if len(ukonczone_gry_true) > 0:
            ukonczone_gry = ukonczone_gry_true
            print("Używam wartości boolean True dla ukończonych gier")
        elif len(ukonczone_gry_string) > 0:
            ukonczone_gry = ukonczone_gry_string
            print("Używam wartości string 'True' dla ukończonych gier")
        else:
            # Jeśli nic nie działa, sprawdź inne możliwe wartości
            print("Nie znaleziono ukończonych gier. Sprawdzam inne możliwe wartości:")
            for val in dane['Track_Completed'].unique():
                count = (dane['Track_Completed'] == val).sum()
                print(f"  Wartość {val} (typ: {type(val).__name__}): {count} wystąpień")
            
            # Jako ostateczność, użyj wszystkich danych
            ukonczone_gry = dane
            print("UWAGA: Używam wszystkich danych bez filtrowania przez Track_Completed")
    else:
        # Jeśli brak kolumny Track_Completed, użyj wszystkich danych
        ukonczone_gry = dane
        print("UWAGA: Brak kolumny Track_Completed, używam wszystkich danych")
    
    # Sprawdź, jaki jest rzeczywisty zakres wartości Track_ID
    if 'Track_ID' in dane.columns:
        print("\nWartości Track_ID w danych:")
        print(dane['Track_ID'].value_counts().sort_index())
    
    # Spróbuj filtrować po torach 1-7, ale bądź elastyczny
    wybrane_tory = [1, 2, 3, 4, 5, 6, 7]
    
    if 'Track_ID' in ukonczone_gry.columns:
        # Sprawdź, czy istnieją tory 1-7
        tory_w_danych = ukonczone_gry['Track_ID'].unique()
        print(f"\nDostępne tory w danych: {sorted(tory_w_danych)}")
        
        # Filtruj tylko wybrane tory
        dane_wybranych_torow = ukonczone_gry[ukonczone_gry['Track_ID'].isin(wybrane_tory)]
        print(f"Liczba gier na torach 1-7: {len(dane_wybranych_torow)}")
        
        # Jeśli nie ma danych dla torów 1-7, użyj dostępnych torów
        if len(dane_wybranych_torow) == 0:
            print("Brak danych dla torów 1-7. Używam dostępnych torów w danych.")
            dane_wybranych_torow = ukonczone_gry
            wybrane_tory = sorted(tory_w_danych)
    else:
        print("UWAGA: Brak kolumny Track_ID, używam wszystkich danych")
        dane_wybranych_torow = ukonczone_gry
    
    # Sprawdź, czy mamy dane
    if len(dane_wybranych_torow) == 0:
        print("Brak danych do analizy po filtrowaniu.")
        return
    
    # Uzyskaj unikalne identyfikatory torów
    unikalne_tory = []
    if 'Track_ID' in dane_wybranych_torow.columns:
        unikalne_tory = dane_wybranych_torow['Track_ID'].unique()
    else:
        # Jeśli nie ma kolumny Track_ID, użyj sztucznych identyfikatorów
        unikalne_tory = wybrane_tory
    
    print(f"Tory do analizy: {sorted(unikalne_tory)}")
    
    # Unikalne relacje i grupy wiekowe (jeśli są dostępne)
    unikalne_relacje = []
    unikalne_grupy_wiekowe = []
    
    if 'Relationship_Name' in dane_wybranych_torow.columns:
        unikalne_relacje = dane_wybranych_torow['Relationship_Name'].unique()
        print(f"Unikalne relacje: {unikalne_relacje}")
    
    if 'Age_Group_X' in dane_wybranych_torow.columns and 'Age_Group_Y' in dane_wybranych_torow.columns:
        grupy_x = dane_wybranych_torow['Age_Group_X'].unique()
        grupy_y = dane_wybranych_torow['Age_Group_Y'].unique()
        unikalne_grupy_wiekowe = np.unique(np.concatenate([grupy_x, grupy_y]))
        print(f"Unikalne grupy wiekowe: {unikalne_grupy_wiekowe}")
    
    # Utwórz katalog na wykresy
    if not os.path.exists('wykresy_weibulla'):
        os.makedirs('wykresy_weibulla')
    
    # Wykonaj testy dla każdego toru
    wyniki = []
    
    for tor in unikalne_tory:
        print(f"\n=== Analiza toru {tor} ===")
        
        if 'Track_ID' in dane_wybranych_torow.columns:
            dane_toru = dane_wybranych_torow[dane_wybranych_torow['Track_ID'] == tor]
        else:
            # Jeśli nie ma kolumny Track_ID, używamy wszystkich danych dla każdego "sztucznego" toru
            dane_toru = dane_wybranych_torow
        
        print(f"Liczba rekordów dla toru {tor}: {len(dane_toru)}")
        
        # Pokaż wartości czasu
        if 'Track_Time' in dane_toru.columns:
            czasy = dane_toru['Track_Time'].dropna()
            czasy_dodatnie = czasy[czasy > 0]
            print(f"Liczba dodatnich czasów dla toru {tor}: {len(czasy_dodatnie)}")
            
            if len(czasy_dodatnie) > 0:
                print(f"Statystyki czasów: min={czasy_dodatnie.min()}, max={czasy_dodatnie.max()}, średnia={czasy_dodatnie.mean()}")
                
                # Test dla wszystkich czasów toru
                wynik = test_weibull(czasy_dodatnie.values, tor)
                if wynik is not None:
                    wyniki.append(wynik)
                
                # Testy dla różnych relacji
                if len(unikalne_relacje) > 0:
                    for relacja in unikalne_relacje:
                        dane_relacji = dane_toru[dane_toru['Relationship_Name'] == relacja]
                        czasy_relacji = dane_relacji['Track_Time'].dropna()
                        czasy_relacji = czasy_relacji[czasy_relacji > 0]
                        
                        if len(czasy_relacji) >= 3:
                            wynik = test_weibull(czasy_relacji.values, tor, relacja, "relationship")
                            if wynik is not None:
                                wyniki.append(wynik)
                
                # Testy dla różnych grup wiekowych
                if len(unikalne_grupy_wiekowe) > 0:
                    for grupa in unikalne_grupy_wiekowe:
                        dane_grupy = dane_toru[(dane_toru['Age_Group_X'] == grupa) | 
                                            (dane_toru['Age_Group_Y'] == grupa)]
                        czasy_grupy = dane_grupy['Track_Time'].dropna()
                        czasy_grupy = czasy_grupy[czasy_grupy > 0]
                        
                        if len(czasy_grupy) >= 3:
                            wynik = test_weibull(czasy_grupy.values, tor, grupa, "age")
                            if wynik is not None:
                                wyniki.append(wynik)
            else:
                print(f"Brak dodatnich czasów dla toru {tor}!")
        else:
            print(f"UWAGA: Brak kolumny Track_Time dla toru {tor}!")
    
    # Zapisz wyniki
    if len(wyniki) > 0:
        wyniki_df = pd.DataFrame(wyniki)
        wyniki_df.to_csv('wyniki_testow_weibulla.csv', index=False)
        print("\nWyniki testów zostały zapisane do pliku 'wyniki_testow_weibulla.csv'")
        return wyniki_df
    else:
        print("Nie znaleziono wystarczających danych do przeprowadzenia testów Weibulla.")
        return None

In [11]:
plik_games = 'games_with_relationship.csv'
rezultaty = wykonaj_testy_weibulla(plik_games)

# Podsumowanie wyników
if rezultaty is not None and not rezultaty.empty:
    print("\n--- Podsumowanie testów Weibulla dla torów 1-7 ---")
    for idx, row in rezultaty.iterrows():
        print(f"Tor {row['tor']}: {'Rozkład Weibulla' if row['is_weibull'] else 'BRAK rozkładu Weibulla'}")

Wczytano plik games_with_relationship.csv z 128179 wierszami

Pierwsze 2 wiersze danych:
              ID      Date  Consent Team_Name  Selected_Language  \
0  20240102-0000  20240102     True    BJ2 00                  0   
1  20240102-0000  20240102     True    BJ2 00                  0   

  Consent_Time_X_axis  Age_X_axis  Companionship_X_axis Question_Time_X_axis  \
0        09:30:48:500        25.0                     1         09:30:55:925   
1        09:30:48:500        25.0                     1         09:30:55:925   

  Consent_Time_Y_axis  ...  Track_Time  Mistake Track_Completed  \
0        09:30:27:247  ...       44594    False            True   
1        09:30:27:247  ...       30630     True           False   

  Interface_Mode                             GAME_ID  Age_Group_X  \
0             []  20240102-0000_BJ200_2_09-31-04-583        18-29   
1             []  20240102-0000_BJ200_4_09-32-10-982        18-29   

   Age_Group_Y  Relationship_X  Relationship_Y  Relatio

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import kstest, lognorm
import os

# Funkcja wczytująca dane
def wczytaj_dane(plik_csv):
    try:
        # Wczytaj dane
        dane = pd.read_csv(plik_csv)
        print(f"Wczytano plik {plik_csv} z {len(dane)} wierszami")
        
        # Upewniamy się, że mamy odpowiednie typy danych
        dane['Track_Time'] = pd.to_numeric(dane['Track_Time'], errors='coerce')
        
        # Konwersja Track_Completed na wartości boolean
        if 'Track_Completed' in dane.columns:
            if dane['Track_Completed'].dtype == 'object':
                dane['Track_Completed'] = dane['Track_Completed'].map({'True': True, 'False': False})
            elif dane['Track_Completed'].dtype in ['int64', 'float64']:
                dane['Track_Completed'] = dane['Track_Completed'] > 0
        
        return dane
    except Exception as e:
        print(f"Błąd wczytywania danych: {e}")
        return None

# Funkcja testująca dopasowanie rozkładu log-normalnego
def test_lognormal(dane_czasowe, nazwa_toru, grupa=None, typ_grupy=None, poziom_istotnosci=0.05):
    if len(dane_czasowe) < 3:
        grupa_info = f", {typ_grupy}: {grupa}" if grupa is not None else ""
        print(f"Za mało danych dla toru {nazwa_toru}{grupa_info} do przeprowadzenia testu.")
        return None
    
    dane_czasowe = np.array(dane_czasowe)
    
    # Estymacja parametrów rozkładu log-normalnego metodą największej wiarygodności
    try:
        # Parametry dla log-normalnego: shape (sigma), loc (mu), scale (exp(mu))
        shape, loc, scale = lognorm.fit(dane_czasowe, floc=0)
        # W przypadku log-normalnego: sigma=shape, mu=np.log(scale)
        mu = np.log(scale)
        sigma = shape
        
        # Test Kołmogorowa-Smirnova
        ks_statistic, p_value = kstest(dane_czasowe, 'lognorm', args=(shape, loc, scale))
        
        # Wartość krytyczna dla testu KS
        critical_value = stats.ksone.ppf(1 - poziom_istotnosci / 2, len(dane_czasowe))
        
        # Czy dane mają rozkład log-normalny?
        is_lognormal = ks_statistic < critical_value
        
        wyniki = {
            'tor': nazwa_toru,
            'grupa': grupa if grupa is not None else "wszystkie",
            'typ_grupy': typ_grupy if typ_grupy is not None else "ogólne",
            'liczba_probek': len(dane_czasowe),
            'mu': mu,
            'sigma': sigma,
            'ks_statistic': ks_statistic,
            'p_value': p_value,
            'critical_value': critical_value,
            'is_lognormal': is_lognormal
        }
        
        # Wydrukuj wyniki
        grupa_info = f", {typ_grupy}: {grupa}" if grupa is not None else ""
        print(f"\n--- Test rozkładu log-normalnego dla toru {nazwa_toru}{grupa_info} ---")
        print(f"Liczba próbek: {len(dane_czasowe)}")
        print(f"Parametry rozkładu: μ={mu:.4f}, σ={sigma:.4f}")
        print(f"Statystyka testu K-S: {ks_statistic:.4f}")
        print(f"Wartość p: {p_value:.4f}")
        print(f"Wartość krytyczna (α={poziom_istotnosci}): {critical_value:.4f}")
        print(f"Wynik testu: {'Dane mają rozkład log-normalny' if is_lognormal else 'Dane NIE mają rozkładu log-normalnego'}")
        
        # Wizualizacja wyników
        katalog = "wykresy_lognormal"
        if typ_grupy is not None:
            katalog = f"{katalog}/tor_{nazwa_toru}/{typ_grupy}"
            os.makedirs(katalog, exist_ok=True)
            nazwa_pliku = f"{katalog}/tor_{nazwa_toru}_{typ_grupy}_{str(grupa).replace(' ', '_').replace('<', 'lt').replace('-', '_')}.png"
        else:
            katalog = f"{katalog}/tor_{nazwa_toru}"
            os.makedirs(katalog, exist_ok=True)
            nazwa_pliku = f"{katalog}/tor_{nazwa_toru}_wszystkie.png"
        
        wizualizuj_lognormal(dane_czasowe, shape, loc, scale, nazwa_toru, grupa, typ_grupy, nazwa_pliku)
        
        return wyniki
    except Exception as e:
        grupa_info = f", {typ_grupy}: {grupa}" if grupa is not None else ""
        print(f"Błąd podczas analizy log-normalnej dla toru {nazwa_toru}{grupa_info}: {e}")
        return None

# Funkcja do wizualizacji dopasowania rozkładu log-normalnego
def wizualizuj_lognormal(dane, shape, loc, scale, nazwa_toru, grupa=None, typ_grupy=None, nazwa_pliku=None):
    plt.figure(figsize=(12, 10))

    title_suffix = f", {typ_grupy}: {grupa}" if grupa is not None else ""

    # Plot 1: Histogram and PDF
    plt.subplot(2, 2, 1)
    counts, bins, _ = plt.hist(dane, bins=max(5, min(20, len(dane)//2)), density=True, alpha=0.7, label='Empirical data')
    x = np.linspace(min(dane), max(dane), 1000)
    pdf = lognorm.pdf(x, shape, loc, scale)
    plt.plot(x, pdf, 'r-', lw=2, label=f'Log-Normal PDF (μ={np.log(scale):.2f}, σ={shape:.2f})')
    plt.title(f'Histogram and PDF - Track {nazwa_toru}{title_suffix}')
    plt.xlabel('Time [ms]')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 2: Empirical and theoretical CDF
    plt.subplot(2, 2, 2)
    sorted_data = np.sort(dane)
    ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    plt.step(sorted_data, ecdf, label='Empirical CDF')
    cdf = lognorm.cdf(x, shape, loc, scale)
    plt.plot(x, cdf, 'r-', lw=2, label='Log-Normal CDF')
    plt.title(f'Cumulative Distribution Function - Track {nazwa_toru}{title_suffix}')
    plt.xlabel('Time [ms]')
    plt.ylabel('Cumulative Probability')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 3: Q-Q Plot
    plt.subplot(2, 2, 3)
    theoretical_quantiles = lognorm.ppf(np.linspace(0.01, 0.99, len(dane)), shape, loc, scale)
    plt.scatter(np.sort(theoretical_quantiles), np.sort(dane))
    min_val = min(min(theoretical_quantiles), min(dane))
    max_val = max(max(theoretical_quantiles), max(dane))
    plt.plot([min_val, max_val], [min_val, max_val], 'k--')
    plt.title(f'Q-Q Plot - Track {nazwa_toru}{title_suffix}')
    plt.xlabel('Theoretical Quantiles')
    plt.ylabel('Empirical Quantiles')
    plt.grid(True, alpha=0.3)

    # Plot 4: P-P Plot
    plt.subplot(2, 2, 4)
    empirical_cdf = ecdf
    theoretical_cdf = lognorm.cdf(sorted_data, shape, loc, scale)
    plt.scatter(empirical_cdf, theoretical_cdf)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'P-P Plot - Track {nazwa_toru}{title_suffix}')
    plt.xlabel('Empirical Probability')
    plt.ylabel('Theoretical Probability')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    if nazwa_pliku:
        plt.savefig(nazwa_pliku)
    plt.close()


# Główna funkcja do przeprowadzenia testów
def wykonaj_testy_lognormal(plik_games):
    # Wczytaj dane
    dane = wczytaj_dane(plik_games)
    
    if dane is None:
        return
    
    # Filtruj tylko ukończone gry lub wszystkie dane, jeśli nie można określić ukończonych
    ukonczone_gry = None
    if 'Track_Completed' in dane.columns:
        # Próba filtrowania z wartością True
        ukonczone_gry_true = dane[dane['Track_Completed'] == True]
        
        if len(ukonczone_gry_true) > 0:
            ukonczone_gry = ukonczone_gry_true
            print(f"Liczba ukończonych gier (Track_Completed == True): {len(ukonczone_gry)}")
        else:
            # Alternatywnie, użyj wszystkich danych
            ukonczone_gry = dane
            print(f"UWAGA: Brak gier z Track_Completed == True. Używam wszystkich {len(dane)} rekordów.")
    else:
        ukonczone_gry = dane
        print(f"UWAGA: Brak kolumny Track_Completed. Używam wszystkich {len(dane)} rekordów.")
    
    # Tory do analizy
    wybrane_tory = [1, 2, 3, 4, 5, 6, 7]
    
    # Filtruj tylko wybrane tory
    dane_wybranych_torow = None
    if 'Track_ID' in ukonczone_gry.columns:
        tory_w_danych = sorted(ukonczone_gry['Track_ID'].unique())
        print(f"Dostępne tory w danych: {tory_w_danych}")
        
        # Filtruj tylko tory 1-7
        dane_torów_1_7 = ukonczone_gry[ukonczone_gry['Track_ID'].isin(wybrane_tory)]
        
        if len(dane_torów_1_7) > 0:
            dane_wybranych_torow = dane_torów_1_7
            print(f"Liczba rekordów dla torów 1-7: {len(dane_wybranych_torow)}")
        else:
            # Jeśli nie ma torów 1-7, użyj wszystkich dostępnych torów
            dane_wybranych_torow = ukonczone_gry
            wybrane_tory = tory_w_danych
            print(f"UWAGA: Brak danych dla torów 1-7. Używam wszystkich dostępnych torów: {tory_w_danych}")
    else:
        dane_wybranych_torow = ukonczone_gry
        print(f"UWAGA: Brak kolumny Track_ID. Używam wszystkich rekordów.")
    
    # Sprawdź, czy mamy dane
    if len(dane_wybranych_torow) == 0:
        print("Brak danych do analizy po filtrowaniu.")
        return
    
    # Uzyskaj unikalne identyfikatory torów do analizy
    unikalne_tory = []
    if 'Track_ID' in dane_wybranych_torow.columns:
        unikalne_tory = sorted(dane_wybranych_torow['Track_ID'].unique())
    else:
        # Jeśli nie ma kolumny Track_ID, używamy sztucznych torów 1-7
        unikalne_tory = wybrane_tory
    
    print(f"Tory do analizy: {unikalne_tory}")
    
    # Unikalne relacje i grupy wiekowe
    unikalne_relacje = []
    unikalne_grupy_wiekowe = []
    
    if 'Relationship_Name' in dane_wybranych_torow.columns:
        unikalne_relacje = sorted(dane_wybranych_torow['Relationship_Name'].unique())
        print(f"Unikalne relacje: {unikalne_relacje}")
    
    if 'Age_Group_X' in dane_wybranych_torow.columns and 'Age_Group_Y' in dane_wybranych_torow.columns:
        grupy_x = dane_wybranych_torow['Age_Group_X'].unique()
        grupy_y = dane_wybranych_torow['Age_Group_Y'].unique()
        unikalne_grupy_wiekowe = np.unique(np.concatenate([grupy_x, grupy_y]))
        print(f"Unikalne grupy wiekowe: {unikalne_grupy_wiekowe}")
    
    # Utwórz katalog na wykresy
    if not os.path.exists('wykresy_lognormal'):
        os.makedirs('wykresy_lognormal')
    
    # Wykonaj testy dla każdego toru
    wyniki = []
    
    for tor in unikalne_tory:
        print(f"\n=== Analiza toru {tor} ===")
        
        # Filtruj dane dla danego toru
        if 'Track_ID' in dane_wybranych_torow.columns:
            dane_toru = dane_wybranych_torow[dane_wybranych_torow['Track_ID'] == tor]
        else:
            # Jeśli nie ma kolumny Track_ID, używamy wszystkich danych
            dane_toru = dane_wybranych_torow
        
        print(f"Liczba rekordów dla toru {tor}: {len(dane_toru)}")
        
        # Pokaż wartości czasu
        if 'Track_Time' in dane_toru.columns:
            czasy = dane_toru['Track_Time'].dropna()
            czasy_dodatnie = czasy[czasy > 0]
            print(f"Liczba dodatnich czasów dla toru {tor}: {len(czasy_dodatnie)}")
            
            if len(czasy_dodatnie) >= 3:
                # Test dla wszystkich czasów toru
                wynik = test_lognormal(czasy_dodatnie.values, tor)
                if wynik is not None:
                    wyniki.append(wynik)
                
                # Testy dla różnych relacji
                if len(unikalne_relacje) > 0:
                    for relacja in unikalne_relacje:
                        dane_relacji = dane_toru[dane_toru['Relationship_Name'] == relacja]
                        if len(dane_relacji) > 0:
                            czasy_relacji = dane_relacji['Track_Time'].dropna()
                            czasy_relacji = czasy_relacji[czasy_relacji > 0]
                            
                            if len(czasy_relacji) >= 3:
                                wynik = test_lognormal(czasy_relacji.values, tor, relacja, "relationship")
                                if wynik is not None:
                                    wyniki.append(wynik)
                
                # Testy dla różnych grup wiekowych
                if len(unikalne_grupy_wiekowe) > 0:
                    for grupa in unikalne_grupy_wiekowe:
                        dane_grupy = dane_toru[(dane_toru['Age_Group_X'] == grupa) | 
                                            (dane_toru['Age_Group_Y'] == grupa)]
                        if len(dane_grupy) > 0:
                            czasy_grupy = dane_grupy['Track_Time'].dropna()
                            czasy_grupy = czasy_grupy[czasy_grupy > 0]
                            
                            if len(czasy_grupy) >= 3:
                                wynik = test_lognormal(czasy_grupy.values, tor, grupa, "age")
                                if wynik is not None:
                                    wyniki.append(wynik)
            else:
                print(f"Za mało danych (< 3) do analizy rozkładu log-normalnego dla toru {tor}")
        else:
            print(f"UWAGA: Brak kolumny Track_Time dla toru {tor}")
    
    # Zapisz wyniki
    if len(wyniki) > 0:
        wyniki_df = pd.DataFrame(wyniki)
        wyniki_df.to_csv('wyniki_testow_lognormal.csv', index=False)
        print("\nWyniki testów zostały zapisane do pliku 'wyniki_testow_lognormal.csv'")
        
        # Podsumowanie wyników
        print("\n--- Podsumowanie testów log-normalnych ---")
        print(f"Przeprowadzono łącznie {len(wyniki_df)} testów")
        
        # Pokaż wyniki testów dla każdego toru
        for tor in wyniki_df['tor'].unique():
            wyniki_toru = wyniki_df[wyniki_df['tor'] == tor]
            print(f"\nTor {tor}:")
            for _, row in wyniki_toru.iterrows():
                grupa_info = f"{row['typ_grupy']}: {row['grupa']}"
                wynik_testu = "Dane mają rozkład log-normalny" if row['is_lognormal'] else "Dane NIE mają rozkładu log-normalnego"
                print(f"  {grupa_info} - {wynik_testu} (próbki: {row['liczba_probek']})")
        
        return wyniki_df
    else:
        print("Nie znaleziono wystarczających danych do przeprowadzenia testów log-normalnych.")
        return None

In [13]:
plik_games = 'games_with_relationship.csv'  # Dostosuj ścieżkę do swojego pliku
wyniki = wykonaj_testy_lognormal(plik_games)

if rezultaty is not None and not rezultaty.empty:
    print("\n--- Podsumowanie testów Weibulla dla torów 1-7 ---")
    for idx, row in rezultaty.iterrows():
        print(f"Tor {row['tor']}: {'Rozkład Weibulla' if row['is_weibull'] else 'BRAK rozkładu Weibulla'}")

Wczytano plik games_with_relationship.csv z 128179 wierszami
Liczba ukończonych gier (Track_Completed == True): 58220
Dostępne tory w danych: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 23]
Liczba rekordów dla torów 1-7: 56271
Tory do analizy: [1, 2, 3, 4, 5, 6, 7]
Unikalne relacje: ['Członek rodziny - członek rodziny', 'Inna znacząca - Inna znacząca', 'Nieznajomy - Nieznajomy', 'To skomplikowane', 'Znajomy - Znajomy']
Unikalne grupy wiekowe: ['18-29' '30-44' '45-59' '60+' '< 18']

=== Analiza toru 1 ===
Liczba rekordów dla toru 1: 7936
Liczba dodatnich czasów dla toru 1: 7936

--- Test rozkładu log-normalnego dla toru 1 ---
Liczba próbek: 7936
Parametry rozkładu: μ=10.4818, σ=0.7326
Statystyka testu K-S: 0.0425
Wartość p: 0.0000
Wartość krytyczna (α=0.05): 0.0152
Wynik testu: Dane NIE mają rozkładu log-normalnego

--- Test rozkładu log-normalnego dla toru 1, relationship: Członek rodziny - członek rodziny ---
Liczba próbek: 3827
Parametry rozkładu: μ=10.5973, σ=0.6778
Statystyka testu